# Proxmox

> Proxmox Virtual Environment (Proxmox VE) is a powerful, open-source platform for managing virtual machines (VMs) and containers. It integrates enterprise-class features with ease of use, making it an excellent choice for virtualization. Here’s an all-in-one guide to help you understand and get started with Proxmox.

- skip_showdoc: true
- skip_exec: true

- Proxmox
  - jupyter docker
  - windows
  - ubuntu
  - pihole
  - home assistant
  - NAS - trueNAS
  - prometheous and grafana
  - blueiris -  camera
 - gpu hdmi pass through for ubuntu

[proxmox setup helper](https://www.youtube.com/watch?v=GoZaMgEgrHw)

## Default Login 

- bthek1 login: root
- password: YME (classic)

### **What is Proxmox?**
- **Type 1 Hypervisor**: Runs directly on hardware for better performance.
- **Based on Debian**: Uses a Linux kernel with KVM (Kernel-based Virtual Machine) and LXC (Linux Containers) for virtualization.
- **Web-Based Management**: Administer everything through an intuitive web interface.
- **Free & Open Source**: Enterprise features available for free; optional support subscription for updates.

### **Key Features**
1. **Virtualization Options**:
   - **KVM**: Full virtualization for running any operating system.
   - **LXC**: Lightweight containers for running Linux-based applications.
2. **High Availability (HA)**:
   - Create clusters with multiple nodes for redundancy and failover.
3. **Backup and Restore**:
   - Built-in backup solutions (full and incremental backups).
   - Supports external storage for backups.
4. **Storage Options**:
   - Local storage, NFS, iSCSI, Ceph, or ZFS file systems.
5. **Networking**:
   - Virtual networking with VLAN support and bonding.
   - Bridged networking for external connectivity.
6. **Proxmox VE Cluster**:
   - Manage multiple nodes from a single interface.
7. **Integrated Web Interface**:
   - Manage VMs, storage, networking, and clusters through a single dashboard.
8. **Flexible Storage**:
   - Mix and match storage technologies (ZFS, Ceph, local disks, etc.).
9. **Live Migration**:
   - Move running VMs between nodes without downtime.
10. **Support for Templates and Clones**:
    - Easily deploy new VMs from existing templates.

### **System Requirements**
- **CPU**: 64-bit processor with virtualization support (Intel VT-x or AMD-V).
- **RAM**: Minimum 2 GB (8 GB or more recommended for multiple VMs).
- **Storage**: SSDs recommended for faster I/O.
- **Network**: At least 1 Gbps for high-performance clusters.

### **Getting Started**
#### 1. **Installation**
- Download the Proxmox ISO from [Proxmox's website](https://www.proxmox.com/).
- Create a bootable USB using tools like Rufus or Etcher.
- Boot from the USB and follow the installation wizard.

#### 2. **Initial Configuration**
- Set up hostname, IP address, and networking.
- Configure storage (local or external).
- Access the Proxmox web interface at `https://<IP_ADDRESS>:8006`.

### **Using Proxmox**
#### 1. **Creating VMs**
- In the web interface, navigate to **Create VM**.
- Configure resources:
  - CPU, RAM, storage, and network interfaces.
- Install the operating system from an ISO image.

#### 2. **Using Containers (LXC)**
- Navigate to **Create CT** for creating lightweight Linux containers.
- Select the Linux distribution template and configure resources.

#### 3. **Storage Management**
- Add storage under **Datacenter > Storage**:
  - Local disks, shared storage (NFS/iSCSI), or distributed storage (Ceph, ZFS).

#### 4. **Backup and Restore**
- Set up backup jobs for VMs and containers.
- Backups can be stored locally or on external storage.

#### 5. **Networking**
- Configure virtual bridges under **Datacenter > Network**.
- Use VLANs for isolated networks or create NAT for external access.

#### 6. **Clustering**
- Add nodes to a cluster for centralized management:
  - Run `pvecm add <IP_ADDRESS>` on new nodes.
- Ensure consistent storage and networking configuration across nodes.

### **Advanced Features**
#### 1. **Proxmox VE Cluster**
- High availability across nodes.
- Shared storage (Ceph or NFS) is recommended for live migration.
  
#### 2. **ZFS Integration**
- Proxmox includes ZFS for advanced storage management:
  - Snapshots, compression, and RAID-like redundancy.

#### 3. **Ceph Storage**
- Use Ceph for scalable, distributed storage:
  - Install Ceph packages and create a Ceph cluster.

#### 4. **PCI Passthrough**
- Pass hardware devices directly to a VM for bare-metal performance.
- Requires a CPU with IOMMU support (Intel VT-d/AMD-Vi).

#### 5. **Cloud-Init**
- Automate VM deployment with preconfigured settings (e.g., hostname, networking).

### **Maintenance**
- **Updates**:
  - Regularly update Proxmox (`apt update && apt upgrade`) for security and feature improvements.
- **Subscriptions**:
  - Optional enterprise subscription for support and access to stable update repositories.
  - Free repositories are available but may include testing features.
- **Logs**:
  - Monitor logs under **Syslog** for troubleshooting.

### **Best Practices**
1. **Use ZFS or Ceph** for resilient storage solutions.
2. **Schedule Regular Backups** of all critical VMs.
3. **Allocate Resources Wisely**:
   - Ensure sufficient CPU and RAM are allocated to each VM or container.
4. **Secure Proxmox**:
   - Use strong passwords and enable two-factor authentication (2FA).
   - Restrict web interface access to trusted IPs.
5. **Networking**:
   - Use VLANs for better isolation and scalability.
6. **Monitor Performance**:
   - Check CPU, memory, and storage usage under the **Dashboard**.

### **Common Issues and Troubleshooting**
1. **VM Won’t Start**:
   - Check if sufficient resources are available.
   - Review logs in `/var/log/pve/tasks`.
2. **Storage Issues**:
   - Ensure correct permissions for shared storage.
3. **Cluster Problems**:
   - Use `pvecm status` to check cluster health.
   - Sync time across nodes using `ntpd` or `chrony`.